In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [2]:
from functions.churn import (
    combine_last_4_cols,
    combine_last_18_cols,
    count_visits,
    get_first_date,
    get_last_date,
    inter_visits,
    calc_date_diff,
    divide_by_x,
    divide_by_y,
    get_pattern
)

In [3]:
df = pd.read_csv("data/Grouped Data.csv")
df['Last 4 Card Digits'] = df['Last 4 Card Digits'].astype(str).str.split('.').str[0].str.zfill(4)

df[['Date', 'Time']] = df['Order Date'].str.split(' ', expand=True)
df['Date'] = pd.to_datetime(df['Date'])

df.drop(columns=['Order Date'], inplace=True)
df = df.sort_values(by='Date')
df.head(2)

,Last 4 Card Digits,Menu Item,Qty,Total,Tip,Date,Time
1234,4220,MODELO,1.0,8.75,10.0,2023-11-05,11:04:00
2390,8834,"Margarita Rock Flight, Elysian Space Dust, Fli...",11.0,162.62,2.0,2023-11-05,12:49:00


In [4]:
df['Month_Year'] = df['Date'].dt.to_period('M')

# Calculate frequency of visits per customer per month
customer_freq = df.groupby(['Last 4 Card Digits', 'Month_Year']).size().unstack(fill_value=0).reset_index()
customer_freq['Monthly Frequency'] = customer_freq.apply(combine_last_4_cols, axis=1)
customer_freq.head(2)

Month_Year,Last 4 Card Digits,2023-11,2023-12,2024-01,2024-02,Monthly Frequency
0,0000,0,0,0,1,"[0, 0, 0, 1]"
1,0004,0,0,1,0,"[0, 0, 1, 0]"


In [5]:
df['Week_Year'] = df['Date'].dt.to_period('W-SUN')

# Calculate frequency of visits per customer per 7th day
customer_freq2 = df.groupby(['Last 4 Card Digits', 'Week_Year']).size().unstack(fill_value=0).reset_index()
customer_freq2['Weekly Frequency'] = customer_freq2.apply(combine_last_18_cols, axis=1)
customer_freq2.head(2)

Week_Year,Last 4 Card Digits,2023-10-30/2023-11-05,2023-11-06/2023-11-12,2023-11-13/2023-11-19,2023-11-20/2023-11-26,2023-11-27/2023-12-03,2023-12-04/2023-12-10,2023-12-11/2023-12-17,2023-12-18/2023-12-24,2023-12-25/2023-12-31,2024-01-01/2024-01-07,2024-01-08/2024-01-14,2024-01-15/2024-01-21,2024-01-22/2024-01-28,2024-01-29/2024-02-04,2024-02-05/2024-02-11,2024-02-12/2024-02-18,2024-02-19/2024-02-25,2024-02-26/2024-03-03,Weekly Frequency
0,0000,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
1,0004,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."


In [6]:
df['Date'] = pd.to_datetime(df['Date'])
grouped_df = df.groupby('Last 4 Card Digits')['Date'].agg(list).reset_index()
grouped_df['Date'] = grouped_df['Date'].apply(lambda x: [date.strftime('%Y-%m-%d') for date in x])

grouped_df['num_visits']   = grouped_df['Date'].apply(count_visits)
grouped_df['first_date']   = grouped_df['Date'].apply(get_first_date)
grouped_df['last_date']    = grouped_df['Date'].apply(get_last_date)
grouped_df['inter_visits'] = grouped_df['Date'].apply(inter_visits)
grouped_df['Date']         = grouped_df['Date'].apply(lambda x: [pd.to_datetime(date) for date in x])

grouped_df['date_diff']  = grouped_df['Date'].apply(calc_date_diff)
grouped_df['division_x'] = grouped_df['date_diff'].apply(lambda x: divide_by_x(x, (customer_freq.shape[1])-1))    # Assuming x is 10
grouped_df['division_y'] = grouped_df['date_diff'].apply(lambda x: divide_by_y(x, (customer_freq2.shape[1])-1))   # Assuming y is 5
grouped_df['Date']       = grouped_df['Date'].apply(lambda x: [date.strftime('%Y-%m-%d') for date in x])

grouped_df.head(2)

,Last 4 Card Digits,Date,num_visits,first_date,last_date,inter_visits,date_diff,division_x,division_y
0,0000,[2024-02-03],1,2024-02-03,2024-02-03,0,0,0.0,0.0
1,0004,[2024-01-14],1,2024-01-14,2024-01-14,0,0,0.0,0.0


In [7]:
combined_df = pd.concat([grouped_df ,customer_freq.iloc[:, -1] , customer_freq2.iloc[:, -1]], axis=1 )
combined_df['Segment'] = combined_df.apply(get_pattern, axis=1)
combined_df.head(2)

,Last 4 Card Digits,Date,num_visits,first_date,last_date,inter_visits,date_diff,division_x,division_y,Monthly Frequency,Weekly Frequency,Segment
0,0000,[2024-02-03],1,2024-02-03,2024-02-03,0,0,0.0,0.0,"[0, 0, 0, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",New
1,0004,[2024-01-14],1,2024-01-14,2024-01-14,0,0,0.0,0.0,"[0, 0, 1, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...",New


In [8]:
lost_df = combined_df[combined_df['Segment'] == 'Lost'].copy()
lost_df.reset_index(drop=True, inplace=True)
lost_df.head(2)

,Last 4 Card Digits,Date,num_visits,first_date,last_date,inter_visits,date_diff,division_x,division_y,Monthly Frequency,Weekly Frequency,Segment
0,0059,[2023-11-18],1,2023-11-18,2023-11-18,0,0,0.0,0.0,"[1, 0, 0, 0]","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Lost
1,0060,[2023-11-18],1,2023-11-18,2023-11-18,0,0,0.0,0.0,"[1, 0, 0, 0]","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Lost


In [9]:
merged_df = pd.merge(df, lost_df, on='Last 4 Card Digits', how='inner')
merged_df = merged_df.iloc[:, :7]
merged_df.head(2)

,Last 4 Card Digits,Menu Item,Qty,Total,Tip,Date_x,Time
0,8834,"Margarita Rock Flight, Elysian Space Dust, Fli...",11.0,162.62,2.00,2023-11-05,12:49:00
1,5709,"HH Mimosa Flight, HH Mimosa Flight, Salad Flig...",4.0,72.08,20.43,2023-11-05,13:58:00


In [ ]:
merged_df.to_csv('data/Churn.csv', index=False)